In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
from IPython.display import clear_output
from DataLoader import CustomDataLoader
from UpNet import UpNet

In [2]:
dataset = CustomDataLoader("res")
net = UpNet()

In [3]:
batch_size = 1
validation_split = .2

dataset_size = dataset.__len__()
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)



In [4]:
# https://pytorch.org/tutorials/beginner/saving_loading_models.html
path_state_dict = "checkpoints/state_dict_01.pth"
path_model = "checkpoints/model_01.pth"

In [5]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    
epoch = 1000
lr = 0.001
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

if train_on_gpu:
    net = net.cuda()

CUDA is available!  Training on GPU ...


In [6]:
def train():
    global net
    total_loss = 0
    net.train()
    count = 0
    for x, y in train_loader:

        if train_on_gpu:
            x = x.cuda()
            y = y.cuda()

        optimizer.zero_grad()
        output = net(x)
        loss = criterion(output, y)
        total_loss += loss.item()
        count+=1
        loss.backward()
        optimizer.step()
    if count > 0 :
        total_loss /= count
    return total_loss
    
    

In [7]:
def test():
    global net
    total_loss = 0
    net.eval()
    count = 0
    with torch.no_grad():
        for x, y in test_loader:
            
            if train_on_gpu:
                x = x.cuda()
                y = y.cuda()
                
            output = net(x)
            loss = criterion(output, y)
            total_loss += loss.item()
            count+=1
    
    if count > 0 :
        total_loss /= count

    return total_loss
    
    

In [8]:

writer = SummaryWriter(comment='Up')

tag_train_loss = "train_loss"
tag_test_loss = "test_loss"


In [9]:
lowest_test_loss = np.inf

for e in range(epoch):
    train_loss = train()
    writer.add_scalar(tag_train_loss, train_loss, e)
    test_loss = test()
    writer.add_scalar(tag_test_loss, test_loss, e)
    
    clear_output()
    print("train loss ",train_loss,"\ttest loss ",test_loss )
    
    if test_loss < lowest_test_loss:
        lowest_test_loss = test_loss
        torch.save(net.state_dict(), path_state_dict)
        torch.save(net, path_model)
        

RuntimeError: CUDA error: unspecified launch failure